In [1]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
import pandas as pd

def load_housing_data(housing_path='data'):
    train_csv_path = os.path.join(housing_path, "train.csv")
    test_csv_path = os.path.join(housing_path, "test.csv")
    return pd.read_csv(train_csv_path), pd.read_csv(test_csv_path)

train, test = load_housing_data()

In [3]:
train['TotalLivingSF'] = train['BsmtFinSF1'] + train['BsmtFinSF2'] + train['1stFlrSF'] + train['2ndFlrSF']
test['TotalLivingSF'] = test['BsmtFinSF1'] + test['BsmtFinSF2'] + test['1stFlrSF'] + test['2ndFlrSF']

train['TotalLivingSF'] = np.log(train['TotalLivingSF'])
test['TotalLivingSF'] = np.log(test['TotalLivingSF'])

In [4]:
train['TotalBthm'] = train['FullBath'] + 0.5*train['HalfBath'] + train['BsmtFullBath'] + 0.5*train['BsmtHalfBath']
test['TotalBthm'] = test['FullBath'] + 0.5*test['HalfBath'] + test['BsmtFullBath'] + 0.5*test['BsmtHalfBath']

train['TotalBthm'] = np.log(train['TotalBthm'])
test['TotalBthm'] = np.log(test['TotalBthm'])

In [5]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
TotalLivingSF    0.698827
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
TotalBthm        0.603681
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorc

In [6]:
num_att = list(train.select_dtypes([np.number]).columns)

# att = ['OverallQual', 'GrLivArea']

# to_remove = ['BsmtFinSF2', 'MoSold', '3SsnPorch', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'YrSold',
#             'PoolArea', 'OverallCond', 'MSSubClass', 'ScreenPorch']

to_remove = ['PoolArea', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'ScreenPorch', '3SsnPorch',
            'BsmtFinSF2', 'MoSold', 'YrSold',
            'GarageArea', 'TotalBsmtSF']

to_remove = ['PoolArea', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'ScreenPorch', '3SsnPorch',
            'BsmtFinSF2', 'MoSold']

for t in to_remove:
    num_att.remove(t)
num_att.remove('Id')
num_att.remove('SalePrice')

# num_att = ['OverallQual', 'GrLivArea', 'GarageCars', '1stFlrSF', 'FullBath', 'YearBuilt']

print(num_att)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'YrSold', 'TotalLivingSF', 'TotalBthm']


In [7]:
cat_att = ['BldgType', 'CentralAir', 'Foundation', 
           'PavedDrive', 'SaleCondition']
cat_att = ['Neighborhood', 'BldgType', 'Heating', 'HeatingQC', 'CentralAir', 
           'Street', 'LotShape', 'LandContour', 'LotConfig',
           'LandSlope', 'Condition1', 'HouseStyle', 'RoofStyle', 
           'ExterQual', 'ExterCond', 'Foundation', 'PavedDrive', 'SaleCondition']

# TO ADD = Exterior1st, KitchenQual, Functional, SaleType, Alley, Condition2

# cat_att = ['Neighborhood', 'BldgType']
print(cat_att)

['Neighborhood', 'BldgType', 'Heating', 'HeatingQC', 'CentralAir', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Condition1', 'HouseStyle', 'RoofStyle', 'ExterQual', 'ExterCond', 'Foundation', 'PavedDrive', 'SaleCondition']


In [8]:
for a in num_att:
    median = train[a].median()
    train[a].fillna(median, inplace=True)
    
#     median = test[a].median()
    test[a].fillna(median, inplace=True)

In [9]:
import seaborn as sns
from scipy.stats import norm
from scipy import stats

train['SalePrice'] = np.log(train['SalePrice'])
train_y = train["SalePrice"].copy()

train['GrLivArea'] = np.log(train['GrLivArea'])
test['GrLivArea'] = np.log(test['GrLivArea'])

train['HasBsmt'] = pd.Series(len(train['TotalBsmtSF']), index=train.index)
train['HasBsmt'] = 0 
train.loc[train['TotalBsmtSF']>0,'HasBsmt'] = 1
train.loc[train['HasBsmt']==1,'TotalBsmtSF'] = np.log(train['TotalBsmtSF'])
test['HasBsmt'] = pd.Series(len(test['TotalBsmtSF']), index=test.index)
test['HasBsmt'] = 0 
test.loc[test['TotalBsmtSF']>0,'HasBsmt'] = 1
test.loc[test['HasBsmt']==1,'TotalBsmtSF'] = np.log(test['TotalBsmtSF'])

train['1stFlrSF'] = np.log(train['1stFlrSF'])
test['1stFlrSF'] = np.log(test['1stFlrSF'])

train['HasBsmtFin'] = pd.Series(len(train['BsmtFinSF1']), index=train.index)
train['HasBsmtFin'] = 0 
train.loc[train['BsmtFinSF1']>0,'HasBsmtFin'] = 1
train.loc[train['HasBsmtFin']==1,'BsmtFinSF1'] = np.log(train['BsmtFinSF1'])
test['HasBsmtFin'] = pd.Series(len(test['BsmtFinSF1']), index=test.index)
test['HasBsmtFin'] = 0 
test.loc[test['BsmtFinSF1']>0,'HasBsmtFin'] = 1
test.loc[test['HasBsmtFin']==1,'BsmtFinSF1'] = np.log(test['BsmtFinSF1'])

train['HasWoodDeck'] = pd.Series(len(train['WoodDeckSF']), index=train.index)
train['HasWoodDeck'] = 0 
train.loc[train['WoodDeckSF']>0,'HasWoodDeck'] = 1
train.loc[train['HasWoodDeck']==1,'WoodDeckSF'] = np.log(train['WoodDeckSF'])
test['HasWoodDeck'] = pd.Series(len(test['WoodDeckSF']), index=test.index)
test['HasWoodDeck'] = 0 
test.loc[test['WoodDeckSF']>0,'HasWoodDeck'] = 1
test.loc[test['HasWoodDeck']==1,'WoodDeckSF'] = np.log(test['WoodDeckSF'])


/home/marek/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.817184
TotalLivingSF    0.741062
GrLivArea        0.730255
GarageCars       0.680625
TotalBthm        0.668685
GarageArea       0.650888
1stFlrSF         0.608947
FullBath         0.594771
YearBuilt        0.586570
YearRemodAdd     0.565608
TotRmsAbvGrd     0.534422
GarageYrBlt      0.495794
Fireplaces       0.489449
MasVnrArea       0.426775
TotalBsmtSF      0.373009
WoodDeckSF       0.343269
LotFrontage      0.335292
OpenPorchSF      0.321053
HasWoodDeck      0.320349
2ndFlrSF         0.319300
HalfBath         0.313982
LotArea          0.257320
BsmtFullBath     0.236224
BsmtUnfSF        0.221985
BedroomAbvGr     0.209044
BsmtFinSF1       0.207962
HasBsmt          0.199634
HasBsmtFin       0.162231
ScreenPorch      0.121208
PoolArea         0.069798
MoSold           0.057329
3SsnPorch        0.054900
BsmtFinSF2       0.004832
BsmtHalfBath    -0.005149
Id              -0.017942
MiscVal         -0.020021
OverallCond     -0.036868
YrSold      

In [11]:
train2 = train[num_att + cat_att]
train2.info()
test2 = test[num_att + cat_att]
test2.info()

merged_df = pd.concat([train2, test2])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 48 columns):
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null float64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null float64
1stFlrSF         1460 non-null float64
2ndFlrSF         1460 non-null int64
GrLivArea        1460 non-null float64
BsmtFullBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
Fireplaces       1460 non-null int64
GarageYrBlt      1460 non-null float64
GarageCars       1460 non-null int64
GarageArea       1460 non-nu

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])
train_prepared = num_pipeline.fit_transform(train[num_att])
test_prepared  = num_pipeline.transform(test[num_att])


full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_att),
        ("cat", OneHotEncoder(), cat_att),
    ])

train_test_full_prepared = full_pipeline.fit_transform(merged_df)
train_full_prepared = full_pipeline.transform(train2)
test_full_prepared = full_pipeline.transform(test2)


In [13]:
print(train_prepared.shape)
print(test_prepared.shape)
print(train_full_prepared.shape)
print(test_full_prepared.shape)

(1460, 30)
(1459, 30)
(1460, 138)
(1459, 138)


In [14]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(bootstrap=False, n_estimators=350, max_features=32, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

housing_predictions = forest_reg.predict(train_full_prepared)

forest_mse = mean_squared_error(train_y, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

0.0014825304385416584

In [15]:
print(housing_predictions[0])
print(train_y[0])

12.247694320220951
12.247694320220994


In [16]:
train_y_normal = np.e**train_y
pred_normal = np.e**housing_predictions

forest_mse = mean_squared_error(train_y_normal, pred_normal)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

207.04715150323068

In [17]:
print(train_y_normal[0])
print(pred_normal[0])

208500.0
208499.9999999911


In [18]:
# forest_reg.feature_importances_

cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = []
for a in cat_encoder.categories_:
    for b in a:
        cat_one_hot_attribs.append(b)
# print(cat_one_hot_attribs)
attributes = num_att + cat_one_hot_attribs
sorted(zip(forest_reg.feature_importances_, attributes), reverse=True)

[(0.1828027301388057, 'OverallQual'),
 (0.12263663657820698, 'GrLivArea'),
 (0.11196871065820235, 'TotalLivingSF'),
 (0.08726102554677743, 'YearBuilt'),
 (0.05466596039882539, 'TA'),
 (0.04424182802127461, 'GarageCars'),
 (0.038611489443972975, 'TotalBthm'),
 (0.03643530378069979, 'GarageArea'),
 (0.035658570207695396, 'TotalBsmtSF'),
 (0.03292284041815341, '1stFlrSF'),
 (0.02451733687590754, 'GarageYrBlt'),
 (0.022477238601088145, 'FullBath'),
 (0.017406023593236414, 'LotArea'),
 (0.01586956048006572, 'Fireplaces'),
 (0.015714644551424614, 'YearRemodAdd'),
 (0.012034398364532297, 'BsmtFinSF1'),
 (0.010992087306638317, '2ndFlrSF'),
 (0.010002285042854948, 'OverallCond'),
 (0.008933304472783414, 'LotFrontage'),
 (0.008857209831878848, 'N'),
 (0.008361406900780277, 'Y'),
 (0.00805284887501053, 'BsmtUnfSF'),
 (0.006679427021515153, 'TotRmsAbvGrd'),
 (0.006203837704417111, 'PConc'),
 (0.0048823656347671265, 'OpenPorchSF'),
 (0.004618454569573804, 'MSSubClass'),
 (0.00454431430610095, 'Gd')

In [19]:
from sklearn.model_selection import cross_val_score
def display_scores(scores):
    print("scores:", scores)
    print("mean:", scores.mean())
    print("std:", scores.std())

forest_reg = RandomForestRegressor(bootstrap=False, n_estimators=200, max_features=32, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

forest_scores = cross_val_score(forest_reg, train_full_prepared, train_y,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

print(pd.Series(forest_rmse_scores).describe())

scores: [0.12833695 0.12068967 0.13252938 0.1702697  0.158876   0.11007637
 0.13179766 0.11558184 0.1321227  0.13341998]
mean: 0.13337002485334581
std: 0.017498615464846164
count    10.000000
mean      0.133370
std       0.018445
min       0.110076
25%       0.122601
50%       0.131960
75%       0.133197
max       0.170270
dtype: float64


In [22]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # sprawdza 12 (3×4) kombinacji hiperparametrów
    {'n_estimators': [100, 300, 400, 500, 600], 'max_features': [16, 32, 48]},
    # następnie sprawdza 6 (2×3) kombinacji z wyłączonym parametrem bootstrap (False)
    {'bootstrap': [False], 'n_estimators': [100, 300, 400, 500, 600], 
     'max_features': [16, 32, 48]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# przeprowadza proces uczenia na pięciu podzbiorach, czyli łącznie (12+6)*5=90 przebiegów 
grid_search = GridSearchCV(forest_reg, param_grid, cv=10,
                           scoring='neg_mean_squared_error',
                           return_train_score=True, verbose=3, n_jobs=-1)
grid_search.fit(train_full_prepared, train_y)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.9min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_features': [16, 32, 48],
   

In [24]:
grid_search.best_params_

{'bootstrap': False, 'max_features': 16, 'n_estimators': 500}

In [25]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
                      max_features=16, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [26]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

0.13848148955154269 {'max_features': 16, 'n_estimators': 100}
0.13695194679226044 {'max_features': 16, 'n_estimators': 300}
0.13682946179226058 {'max_features': 16, 'n_estimators': 400}
0.13669734190228536 {'max_features': 16, 'n_estimators': 500}
0.13685269090583774 {'max_features': 16, 'n_estimators': 600}
0.13713386574759961 {'max_features': 32, 'n_estimators': 100}
0.136151474580309 {'max_features': 32, 'n_estimators': 300}
0.13592470561499959 {'max_features': 32, 'n_estimators': 400}
0.1356867774597488 {'max_features': 32, 'n_estimators': 500}
0.13567097735753478 {'max_features': 32, 'n_estimators': 600}
0.13622225922255923 {'max_features': 48, 'n_estimators': 100}
0.1354385337368978 {'max_features': 48, 'n_estimators': 300}
0.13551165692412306 {'max_features': 48, 'n_estimators': 400}
0.1355033612048938 {'max_features': 48, 'n_estimators': 500}
0.1356286964477476 {'max_features': 48, 'n_estimators': 600}
0.13491832687942465 {'bootstrap': False, 'max_features': 16, 'n_estimators':

In [27]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([5.88505065e-03, 1.25022384e-02, 2.14090671e-02, 1.15310313e-01,
       9.81259745e-03, 6.39892027e-02, 2.49032745e-02, 8.47246914e-03,
       1.77119358e-02, 9.62457630e-03, 4.31757469e-02, 3.51891809e-02,
       1.65309127e-02, 8.77636556e-02, 1.95749726e-03, 3.78913518e-02,
       3.41205600e-03, 5.05985183e-03, 2.21227296e-03, 1.40382696e-02,
       1.86056452e-02, 3.01698572e-02, 4.71854500e-02, 4.88179632e-02,
       4.39560708e-03, 1.03203373e-02, 2.06557833e-03, 2.58956191e-03,
       9.51500008e-02, 3.82766355e-02, 6.44711489e-05, 5.28888594e-06,
       2.08653854e-04, 4.51503720e-04, 5.42644823e-04, 4.27132339e-04,
       1.82723711e-03, 1.60290443e-03, 2.35208900e-04, 1.13746046e-03,
       5.16443011e-04, 2.12211011e-04, 8.35424287e-04, 3.60581967e-05,
       4.17032674e-04, 4.28180838e-04, 2.54580408e-03, 1.40335907e-03,
       2.06004587e-04, 5.08571859e-04, 2.28882477e-04, 4.38042836e-04,
       3.34649762e-04, 2.09026306e-04, 1.03337169e-04, 2.03127223e-03,
      

In [28]:
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = []
for a in cat_encoder.categories_:
    for b in a:
        cat_one_hot_attribs.append(b)
# print(cat_one_hot_attribs)
attributes = num_att + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.11531031306211151, 'OverallQual'),
 (0.09515000083536287, 'TotalLivingSF'),
 (0.08776365560233458, 'GrLivArea'),
 (0.06398920269880165, 'YearBuilt'),
 (0.0522040076126163, 'TA'),
 (0.048817963191655774, 'GarageArea'),
 (0.04718544998538426, 'GarageCars'),
 (0.04317574690152887, 'TotalBsmtSF'),
 (0.03827663550146994, 'TotalBthm'),
 (0.03789135176943227, 'FullBath'),
 (0.035189180931426776, '1stFlrSF'),
 (0.030169857199539088, 'GarageYrBlt'),
 (0.024903274500156006, 'YearRemodAdd'),
 (0.021409067109807604, 'LotArea'),
 (0.018605645217304064, 'Fireplaces'),
 (0.01771193582629896, 'BsmtFinSF1'),
 (0.016530912722128643, '2ndFlrSF'),
 (0.01527280742743347, 'PConc'),
 (0.014038269552204144, 'TotRmsAbvGrd'),
 (0.012502238379212088, 'LotFrontage'),
 (0.010320337270195542, 'OpenPorchSF'),
 (0.010030355254696336, 'Gd'),
 (0.009812597452942624, 'OverallCond'),
 (0.009624576297537527, 'BsmtUnfSF'),
 (0.008472469136237469, 'MasVnrArea'),
 (0.008146589523237343, 'Y'),
 (0.007326215321563303, 'N')

In [29]:
# y_pred = forest_reg.predict(test_full_prepared)
y_pred = grid_search.predict(test_full_prepared)
y_pred_normal = np.e**y_pred
sub = pd.DataFrame({'Id': test['Id'], 'SalePrice': y_pred_normal})

In [30]:
sub.to_csv("data/submission_script30_TotalBthm.csv", index=False)

In [31]:
sub.shape

(1459, 2)